In [1]:
!git clone https://github.com/nkt780426/Bachelor-s-Project.git

Cloning into 'Bachelor-s-Project'...
remote: Enumerating objects: 524, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 524 (delta 24), reused 88 (delta 23), pack-reused 408 (from 1)
Receiving objects: 100% (524/524), 76.95 MiB | 41.98 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [2]:
!mv Bachelor-s-Project/going_modular .

In [3]:
!ls -la

total 48
drwxr-xr-x 4 root root  4096 Dec 19 18:10 .
drwxr-xr-x 6 root root  4096 Dec 19 18:09 ..
drwxr-xr-x 5 root root  4096 Dec 19 18:10 Bachelor-s-Project
drwxr-xr-x 7 root root  4096 Dec 19 18:10 going_modular
---------- 1 root root 29294 Dec 19 18:10 __notebook__.ipynb


In [4]:
import warnings
warnings.filterwarnings("ignore")
import os

import torch
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import albumentations as A

from going_modular.dataloader.magface import create_magface_dataloader
from going_modular.model.MagFaceRecognition import MagFaceRecognition
from going_modular.train_eval.magface.train import fit
from going_modular.loss.MagLoss import MagLoss
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint
from going_modular.utils.transforms import RandomResizedCropRect, GaussianNoise

device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Đặt seed toàn cục
seed = 42
torch.manual_seed(seed)

CONFIGURATION = {
    # Thư mục
    'type': 'normalmap',
    'train_dir': '/kaggle/input/hoangvn-3dfacerecognition/Normal_Map/train',
    'test_dir': '/kaggle/input/hoangvn-3dfacerecognition/Normal_Map/test',
    
    # Cấu hình train
    'backbone': 'iresnet18',
    'epochs': 500,
    'num_workers': 4,
    'batch_size': 16,
    'image_size': 224,
    'num_class': len(os.listdir('/kaggle/input/hoangvn-3dfacerecognition/Normal_Map/train')),
    'embedding_size': 512,
    
    'learning_rate': 0.2,
    'weight_decay': 5e-4,
    'momentum': 0.9,
    'alpha': 0.9,
    
    # Hàm m(ai) giúp thay đổi ai từ 0.25 đến 1.6
    'scale': 64,
    'lambda_g': 20,
    'l_margin': 0.45, 
    'u_margin': 0.8,
    'l_a': 10, 
    'u_a': 110,
}

train_transform = A.Compose([
    RandomResizedCropRect(256),
    GaussianNoise(),
])

test_transform = A.Compose([
    A.Resize(height=CONFIGURATION['image_size'], width=CONFIGURATION['image_size'])
])

train_dataloader, test_dataloader = create_magface_dataloader(CONFIGURATION, train_transform, test_transform)

model = MagFaceRecognition(CONFIGURATION).to(device)
criterion = MagLoss(conf = CONFIGURATION)

optimizer = torch.optim.Adam(model.parameters(), lr=CONFIGURATION['learning_rate'])
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=40, T_mult=1, eta_min=1e-6)
checkpoint_path = os.path.abspath('checkpoint/magface/' + CONFIGURATION['type'] + '/models/checkpoint.pth')
modle_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1)
earlystop_dir = os.path.abspath('checkpoint/magface/' + CONFIGURATION['type'] + '/models')
early_stopping = MultiMetricEarlyStopping(
    monitor_keys=['test_cosine_auc', 'test_cosine_accuracy', 'test_euclidean_auc', 'test_euclidean_accuracy'],
    patience=40,
    mode='max',
    verbose=1,
    save_dir=earlystop_dir,
    start_from_epoch=40
)            

In [5]:
fit(CONFIGURATION, 0, model, device, train_dataloader, test_dataloader, criterion, optimizer, scheduler, early_stopping, modle_checkpoint)

Epoch 1:
	train: loss 3.528 | loss id   3.50 | top_1_acc 0.0034 | top_5_acc 0.0225 | cos_auc: 0.5335 | cos_acc: 0.5253 | eu_auc: 0.5386 | eu_acc: 0.6713
	test: cos_auc: 0.5456 | cos_acc: 0.4366 | eu_auc: 0.5362 | eu_acc: 0.7221
	Saving model and optimizer state to /kaggle/working/checkpoint/magface/normalmap/models/checkpoint.pth
Epoch 2:
	train: loss 1.819 | loss id   1.80 | top_1_acc 0.0045 | top_5_acc 0.0236 | cos_auc: 0.5646 | cos_acc: 0.7748 | eu_auc: 0.5655 | eu_acc: 0.8697
	test: cos_auc: 0.6100 | cos_acc: 0.8429 | eu_auc: 0.6079 | eu_acc: 0.7757
	Saving model and optimizer state to /kaggle/working/checkpoint/magface/normalmap/models/checkpoint.pth
Epoch 3:
	train: loss 0.566 | loss id   0.54 | top_1_acc 0.0045 | top_5_acc 0.0281 | cos_auc: 0.5948 | cos_acc: 0.5316 | eu_auc: 0.5757 | eu_acc: 0.4886
	test: cos_auc: 0.6053 | cos_acc: 0.4477 | eu_auc: 0.5822 | eu_acc: 0.4084
	Saving model and optimizer state to /kaggle/working/checkpoint/magface/normalmap/models/checkpoint.pth
Epoc